Benefit of being able to help people sift through myriad availibilities. 

# DATA UNDERSTANDING

What does the data look like?

What information can we glean?

In [1]:
# from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import percent_rank
from pyspark.ml.recommendation import ALS

In [2]:
import pyspark
from pyspark.sql import SQLContext
from pyspark import SparkConf

conf = SparkConf()
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '1G')
        .set('spark.driver.memory', '10G')
        .set('spark.driver.maxResultSize', '5G'))
sc = pyspark.SparkContext(conf=conf)

sql_context = SQLContext(sc)

^^Upped the memory to try to prevent java heap space OutOfMemory errors

In [3]:
df = sql_context.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('data/ratings.csv')

In [4]:
df.count()

26024289

In [5]:
df.columns

['userId', 'movieId', 'rating', 'timestamp']

In [6]:
df

DataFrame[userId: int, movieId: int, rating: double, timestamp: int]

In [7]:
df.take(3)

[Row(userId=1, movieId=110, rating=1.0, timestamp=1425941529),
 Row(userId=1, movieId=147, rating=4.5, timestamp=1425942435),
 Row(userId=1, movieId=858, rating=5.0, timestamp=1425941523)]

#### Train Test Split by date at 10 most recent percent of the data

In [8]:
time_ranked_df = df.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("timestamp")))

In [9]:
train_df = time_ranked_df.where('rank <= 0.9').drop('rank')
test_df = time_ranked_df.where('rank > 0.9').drop('rank')

In [10]:
train_df.persist()
test_df.persist()

DataFrame[userId: int, movieId: int, rating: double, timestamp: int]

In [11]:
f"""
Train rows: {train_df.count()}
Test rows: {test_df.count()}
"""

'\nTrain rows: 23421860\nTest rows: 2602429\n'

#### Sanity Test: On this processor how feasable is ALS model on 23 mil rows

In [12]:
als = ALS(userCol='userId' ,
          itemCol='movieId' ,
          ratingCol='rating'
         )

In [15]:
# sc.setCheckpointDir('/tmp')   # Early attempt to avoid Out Of Memory errors

In [14]:
model = als.fit(train_df)

In [ ]:
als

# DATA PREPARATION

Train Test Split

# MODELING

PySpark Alternating Least Squares

# EVALUATION

Would evaluate in ... manner.

# DEPLOYMENT

Would deploy in ... manner.